# LINEAR REGRESSION
## Linear Regression apllied to a Google stokes avaloble on Quandl

The code use the stokes price avalible on quandl and use this data to fit a linear regression model.

First we need to define the features, on this case we took as features the Adj. Close, Adj. Volume and we define two other features HIGH_LOW_PERCENT (HL_PCT) and PERCENT_CHANGE (PCT_CHANGE) this features give us some idea abou the volatility of the stokes. This features are not the best ones but this example should be simple.

Next we need to slip the data in pieces, the first piece will be our training data on this data we will fit ou linear reegression model (hyphothesis). The second piece will be our test data on thsi data we will see how accuracy is our hypothesis, and the last piece is the data use to forecast. Them we scale the data

The library SKLEARN offers all the functions that we need to computate our model 

To plot our data we need to transformat the index of the dataframe to be a dtype = datetime. 

In [1]:
%matplotlib ##PLOT IN A DIFFERENT WINDOW

Using matplotlib backend: Qt5Agg


In [15]:
################################### LINEAR REGRESSION ###########################################
## GUILHERME CARVALHO PEREIRA
## IMPORTING LIBARIES
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn import linear_model
import sklearn.preprocessing
import math,time, quandl
from datetime import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

## DEFINING THE STYLE OF THE PLOTS
style.use('ggplot')
## GETING THE DATE SET FORM QUANDL
quandl.ApiConfig.api_key = 'G9SiyZ49oi4T_KYraTQs'
df = quandl.get('WIKI/GOOGL', index_col='Date', parse_dates=True)
## PREVIEWING THE DATA
#print(df_raw.head())
## GETING THE FEATURES MORE MEANINGFUL
df = df[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]
#################################################################################################
## DEFINING THE SPECIAL RELATIONSHIP BETWEEN THE FEATURES
## HIGH_LOW_PERCENT (HL_PCT) => THIS FEATURE GIVE US THE PERCENT VOLATILITY OF THE STOCKS
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close'])/df['Adj. Close'] *100 
## PERCENT_CHANGE (PCT_CHANGE) => THIS FEATURE GIVE US THE DAILY MOVE OF THE STOCKS
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open']*100.0
## DEFINING THE DATAFRAME WITH THE SPECIAL RELATIONSHIPS
df = df[['Adj. Close','HL_PCT','PCT_change','Adj. Volume']]
## PREVIEWING THE DATA
#print(df.head())
## DEFINING THE LABEL... THE 'THING' WE WANT TO PREDICT
forecast_col = 'Adj. Close'
## FILLING THE NaN DATA ON THE DATASET
df.fillna(-99999, inplace=True)
## GETING 10% OF THE DATAFRAME AND TRY TO PREDICT
pct_data = 0.01  ##GIVE HOW MUCH OF THE DATASET WE ARE TRYING TO PREDICT
forecast_out = int(math.ceil(pct_data*len(df)))
## ADD THE LABEL TO THE DATAFRAME
df['Label'] = df[forecast_col].shift(-forecast_out)  ##JUST THE COLLUMS 10 DAYS ON THE FUTURE
## PREVIEWING THE DATA
#print(df.head())
## DEFINING X AND y FOR OUR HYPOTHESIS
X = np.array(df.drop(['Label'], 1))
## SCALING OUR FEATURES
X = sklearn.preprocessing.scale(X)
## DEFINING THE DATA FRO THE FORECAST
X_lately = X[-forecast_out:]  ##DATA TO FORECAST
X = X[:-forecast_out]
## DEVIDING OUR DATA INTO TRAINING AND DATA SET...TESTE SIZE IN %
df.dropna(inplace = `True`)
y = np.array(df['Label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
## DEFINING OUR LINEAR REGRESSION HYPOTHESIS
hypo = linear_model.LinearRegression()
hypo.fit(X_train, y_train)
accuracy = hypo.score(X_test, y_test)
##print'THIS ALGORITHM PREDICTS THE OUTPUT WITH A ACCURACY OF {}%'.format(accuracy*100)
## PREDICTION THE NEXT PRICE OF THE STOCKS
y_predict = hypo.predict(X_lately)
#################################################################################################
## PREPARING THE DATA SET TO BE PLOT
df['Forecast'] = np.nan
df.index
last_date = df.iloc[-1].name
last_unix = time.mktime(time.strptime(str(last_date), "%Y-%m-%d %H:%M:%S")) # .timestamp()
one_day = 86400
next_unix = last_unix + one_day

for i in y_predict:
    next_date = dt.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)] + [i]
##PLOTING THE DATA AND THE PREDICTION
df['Adj. Close'].plot()
df['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

## BUILDING A LINEAR REGRESSION FROM SCRATCH

The code we try to build a linear regression model from scratch, we define function to create a total random dataset and than apply thsi random data to a function to compute the best fit line for data (compute m, the slope, and b, the bias). Further we aplicate a squared error the for the linear model, because this should be a simple exemple we didn't need to feedback the error to our best fit line (keep in mind that this should be the normal standard). On he end we plot our data and our linear model.

In [38]:
########################## LINEAR REGRESSION FUNCTION FROM SCRATCH ###########################
from statistics import mean
import numpy as np
import random 
import matplotlib.pyplot as plt
from matplotlib import style
## DEFINING THE STYLE OF THE PLOTS
style.use('ggplot')

''' THIS FUNCTION CREATE A SUDO RADOM DATASET 
    THE FUNCTION TAKES FOUR PARAMETERS:
    size -> THE NUMBER OF DATA POINTS TO BE GENERATE
    variance -> THE VARIANCE OF THE DATESET
    step -> THE INCREMENET OF THE DATASET. BY DEFAULT step=2
    correlation ->THE THE DATA POINTS HAVE ANY CORRELATION. BY DEFAULT correlation = False
    
    THE FUNCTION RETURNS TWO NUMPY ARRAYS, X AND Y, dtype=np.float64
'''
def create_dateset(size, variance, step=2, correlation=False):
    val = 1
    y = []
    for i in range(size):
        y_temp = val+random.randrange(-variance,variance)
        y.append(y_temp)
        
        if correlation and correlation == 'pos':
            val+=step
            
        elif correlation and correlation == 'neg':
            val-=step
            
    X = [i for i in range(size)]
    
    return np.array(X, dtype = np.float64), np.array(y, dtype = np.float64)

''' THIS FUNCTION CALCULATES THE BEST FIT LINE TO A REGRESSION MODEL
    THE FUNCTION TAKES TWO PARAMETES:
    X -> DATA TO BE FIT AS THE x AXIS : DATA TYPE => numpy.array
    y -> DATA TO BE FIT AS THE y AXIS : DATA TYPE => numpy.array
    
    THE FUNCTION RETURNS TWO FLOAT NUMBERS, m AND b
'''
def best_fit_line(X,y):
    ## COMPUTATING m
    m = (((mean(X)*mean(y)) - mean(X*y))/
        ((mean(X)**2)-mean(X**2)))
    b = mean(y) - m*mean(X)
    ## RETURNING THE SLOPE AND INTERCEPT
    return m, b


''' THIS FUNCTION CALCULATES THE SQUARED ERROR TO A REGRESSION MODEL
    THE FUNCTION TAKES TWO PARAMETES:
    y -> DATA TO BE FIT AS THE y AXIS : DATA TYPE => numpy.array
    regression_model -> DATA PREDICT USING THE best_fit_line function : DATA TYPE => numpy.array
    
    THE FUNCTION RETURN ONE FLOAT NUMBER
'''
def squared_error(y,regression_model):
    return sum((regression_model - y) * (regression_model - y))

''' THIS FUNCTION CALCULATES THE COEFICIENT OF DETERMINATION TO A REGRESSION MODEL
    THIS COEFICIENT TELLS US HOW "GOOD" IS THE REGRESSION MODEL...GOOD MODEL OR NOT
    A GOOD COEFICENT VALUE IS THE CLOSE TO 1 AS POSSIBLE
    THE FUNCTION TAKES TWO PARAMETES:
    y -> DATA TO BE FIT AS THE y AXIS : DATA TYPE => numpy.array
    regression_model -> DATA PREDICT USING THE best_fit_line function : DATA TYPE => numpy.array
    
    THE FUNCTION RETURN ONE FLOAT NUMBER
'''
def coef_of_determination(y, regression_model):
    y_mean = [mean(y) for y_line in y]
    
    squared_error_regression = squared_error(y,regression_model)
    squared_error_mean = squared_error(y, y_mean)
    
    #print y, regression_model, y_mean
    print squared_error_regression, squared_error_mean
    return 1-(squared_error_regression/squared_error_mean)

## TESTING DATA

X, y = create_dateset(40,40,2,correlation='pos') 


## CALLING THE FUNCTION

[m,b] = best_fit_line(X,y)

print m, b

## LINE FIT

regression_model = [(m*x)+b for x in X]

## TESTING HOW GOOD THE LINE FITS THIS DATA

r_squared = coef_of_determination(y, regression_model)
print r_squared

## PLOTING THE DATA
plt.scatter(X,y)
plt.plot(X,regression_model , label='LINEAR_REGRESSION', color='b')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

1.7679174484052527 1.6256097560975746
18338.5138836773 34997.6
0.4760065294855276
